<a href="https://colab.research.google.com/github/hondore16/PIONS/blob/main/tran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Практика 1: Классификация текста с помощью трансформеров
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Инициализация устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка датасета и токенизатора
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Подготовка модели
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

# Токенизация данных
tokenized_datasets = dataset.map(
    lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True),
    batched=True
)

# Создание уменьшенных выборок
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Настройка обучения
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="no",
    logging_dir='./logs',
    logging_steps=50,
    seed=42,
)
training_args.evaluation_strategy = "epoch"
# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer
)

# Запуск обучения и оценки
trainer.train()
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-3b1341386d57>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.656500
100,0.432400
150,0.328300
200,0.283600
250,0.320000
300,0.189800
350,0.095600
400,0.144900
450,0.054600
500,0.075800


{'eval_loss': 0.636085569858551,
 'eval_runtime': 30.1186,
 'eval_samples_per_second': 33.202,
 'eval_steps_per_second': 4.15,
 'epoch': 5.0}

In [ ]:
# Практика 2: Реализация механизма внимания
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.q_proj = torch.nn.Linear(embed_dim, embed_dim)
        self.k_proj = torch.nn.Linear(embed_dim, embed_dim)
        self.v_proj = torch.nn.Linear(embed_dim, embed_dim)
        self.out_proj = torch.nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        Q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1,2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn = torch.nn.functional.softmax(scores, dim=-1)

        out = torch.matmul(attn, V)
        out = out.transpose(1,2).contiguous().view(batch_size, seq_len, -1)
        return self.out_proj(out)

# Практика 3: Полная реализация трансформера
class TransformerEncoder(torch.nn.Module):
    def __init__(self, num_layers, d_model, num_heads, ff_dim, vocab_size, max_len=512):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.layers = torch.nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, ff_dim)
            for _ in range(num_layers)
        ])

    def forward(self, src):
        src = self.embedding(src) * (self.embedding.embedding_dim ** 0.5)
        src = self.pos_encoding(src)
        for layer in self.layers:
            src = layer(src)
        return src

class TransformerEncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, ff_dim):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.linear1 = torch.nn.Linear(d_model, ff_dim)
        self.linear2 = torch.nn.Linear(ff_dim, d_model)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)

    def forward(self, src):
        src = self.norm1(src + self.self_attn(src))
        src = self.norm2(src + self.linear2(torch.nn.functional.relu(self.linear1(src))))
        return src

class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, max_len):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(1, max_len, d_model)
        self.pe[0, :, 0::2] = torch.sin(position * div_term)
        self.pe[0, :, 1::2] = torch.cos(position * div_term)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [12]:
import torch
import torch.nn.functional as F

def predict_sentiment(texts, model, tokenizer, device):
    model.eval()
    # Токенизация
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Перемещаем входные данные на устройство модели (GPU или CPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

    return preds.cpu().numpy(), probs.cpu().numpy()

# Пример использования
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")

# Предполагается, что model и tokenizer уже загружены
model.to(device)

texts = [
    "This movie was fantastic! I really enjoyed it.",
    "Terrible film. Waste of time.",
    "It was okay, not the best but not the worst."
]

preds, probs = predict_sentiment(texts, model, tokenizer, device)

for text, pred, prob in zip(texts, preds, probs):
    label = "Positive" if pred == 1 else "Negative"
    confidence = prob[pred]
    print(f"Text: {text}\nPrediction: {label} (confidence: {confidence:.2f})\n")

Используем устройство: cuda
Text: This movie was fantastic! I really enjoyed it.
Prediction: Positive (confidence: 1.00)

Text: Terrible film. Waste of time.
Prediction: Negative (confidence: 1.00)

Text: It was okay, not the best but not the worst.
Prediction: Negative (confidence: 0.99)



In [13]:
# Получаем первые 10 примеров
test_samples = small_eval_dataset.select(range(100))

# Извлекаем тексты из токенизированных input_ids
texts = [tokenizer.decode(x['input_ids'], skip_special_tokens=True) for x in test_samples]

# Извлекаем метки
labels = [x['label'] for x in test_samples]

# Определяем устройство (GPU, если доступен)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Перемещаем модель на устройство

# Предсказания модели с передачей устройства
preds, probs = predict_sentiment(texts, model, tokenizer, device)

# Вывод результатов
for text, true_label, pred, prob in zip(texts, labels, preds, probs):
    true_str = "Positive" if true_label == 1 else "Negative"
    pred_str = "Positive" if pred == 1 else "Negative"
    confidence = prob[pred]
    print(f"Text: {text}\nTrue: {true_str}, Predicted: {pred_str} (confidence: {confidence:.2f})\n")

Text: < br / > < br / > when i unsuspectedly rented a thousand acres, i thought i was in for an entertaining king lear story and of course michelle pfeiffer was in it, so what could go wrong? < br / > < br / > very quickly, however, i realized that this story was about a thousand other things besides just acres. i started crying and couldn ' t stop until long after the movie ended. thank you jane, laura and jocelyn, for bringing us such a wonderfully subtle and compassionate movie! thank you cast, for being involved and portraying the characters with such depth and gentleness! < br / > < br / > i recognized the angry sister ; the runaway sister and the sister in denial. i recognized the abusive husband and why he was there and then the father, oh oh the father... all superbly played. i also recognized myself and this movie was an eye - opener, a relief, a chance to face my own truth and finally doing something about it. i truly hope a thousand acres has had the same effect on some othe